In [16]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import random_split
transform_Training = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(size=32),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
    transforms.RandomRotation(10),
])
transform_Testing = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_Training)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_Testing)

Files already downloaded and verified
Files already downloaded and verified


In [17]:
print(len(train_dataset))
print(len(test_dataset))

50000
10000


In [18]:
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=10, shuffle=True)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=10, shuffle=False)
classes=( 'plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship','truck')

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.conv5 = nn.Conv2d(256, 512, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(512 * 2 * 2, 1024)  
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = self.pool(F.relu(self.conv5(x)))
        
        x = x.view(-1, 512 * 2 * 2)  
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = CNN()
inputs = torch.randn(10, 3, 32, 32)  
outputs = model(inputs)  
print(outputs.size())  


torch.Size([10, 10])


In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.005)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=2048, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=10, bias=True)
)

In [21]:
num_epochs = 25
for epoch in range(num_epochs):
    model.train()  
    running_loss = 0.0

    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(trainloader)}")

print('Finished Training')


Epoch 1, Loss: 2.3026292129039763
Epoch 2, Loss: 2.299273066854477
Epoch 3, Loss: 2.1075579008817673
Epoch 4, Loss: 1.9338271827220916
Epoch 5, Loss: 1.7895488530755044
Epoch 6, Loss: 1.6611233338236808
Epoch 7, Loss: 1.5712940850377082
Epoch 8, Loss: 1.494913510143757
Epoch 9, Loss: 1.423121839594841
Epoch 10, Loss: 1.3592015258789063
Epoch 11, Loss: 1.298225670748949
Epoch 12, Loss: 1.2400246995210649
Epoch 13, Loss: 1.1838270207464694
Epoch 14, Loss: 1.1308986563712358
Epoch 15, Loss: 1.084954586753249
Epoch 16, Loss: 1.0410763949602841
Epoch 17, Loss: 1.0017124828755855
Epoch 18, Loss: 0.9640339728534222
Epoch 19, Loss: 0.9322494791045784
Epoch 20, Loss: 0.9014462029218674
Epoch 21, Loss: 0.8724125379130244
Epoch 22, Loss: 0.8426047848373651
Epoch 23, Loss: 0.8187734300196171
Epoch 24, Loss: 0.7913735871732235
Epoch 25, Loss: 0.7695870751809328
Finished Training


In [23]:
start_epoch = 25
end_epochs = 35
for epoch in range(start_epoch, end_epochs, 1):
    model.train()  
    running_loss = 0.0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(trainloader)}")

print('Finished Training')


Epoch 26, Loss: 0.7462742319099606
Epoch 27, Loss: 0.721329242747277
Epoch 28, Loss: 0.704905159368366
Epoch 29, Loss: 0.6821159638669342
Epoch 30, Loss: 0.6586104024771601
Epoch 31, Loss: 0.640824383200705
Epoch 32, Loss: 0.6230413900088518
Epoch 33, Loss: 0.6042710888631642
Epoch 34, Loss: 0.5877361650690436
Epoch 35, Loss: 0.5723388158153743
Finished Training


In [25]:
start_epoch = 35
end_epochs = 50
for epoch in range(start_epoch, end_epochs, 1):
    model.train()  
    running_loss = 0.0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(trainloader)}")

print('Finished Training')


Epoch 36, Loss: 0.5553142989493907
Epoch 37, Loss: 0.533291467897594
Epoch 38, Loss: 0.524412017734535
Epoch 39, Loss: 0.5058946604644879
Epoch 40, Loss: 0.48695079673416913
Epoch 41, Loss: 0.4754234429757111
Epoch 42, Loss: 0.4654476029738784
Epoch 43, Loss: 0.4512816393664107
Epoch 44, Loss: 0.43175039503611623
Epoch 45, Loss: 0.42052178689138964
Epoch 46, Loss: 0.40440464558098466
Epoch 47, Loss: 0.3969259930463508
Epoch 48, Loss: 0.384083883972466
Epoch 49, Loss: 0.370896361504728
Epoch 50, Loss: 0.3583421970269177
Finished Training


In [27]:
start_epoch = 50
end_epochs = 65
for epoch in range(start_epoch, end_epochs, 1):
    model.train()  
    running_loss = 0.0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(trainloader)}")

print('Finished Training')


Epoch 51, Loss: 0.35512890000543557
Epoch 52, Loss: 0.3416333910762798
Epoch 53, Loss: 0.3294113023949787
Epoch 54, Loss: 0.31971327293501234
Epoch 55, Loss: 0.3078098337261006
Epoch 56, Loss: 0.3038333537615603
Epoch 57, Loss: 0.29243055733377116
Epoch 58, Loss: 0.2872379967159592
Epoch 59, Loss: 0.2773678955581971
Epoch 60, Loss: 0.26607771515832285
Epoch 61, Loss: 0.2640403755318606
Epoch 62, Loss: 0.26391682389436755
Epoch 63, Loss: 0.2501098977719666
Epoch 64, Loss: 0.24350587415485642
Epoch 65, Loss: 0.2330261222324567
Finished Training


In [28]:
start_epoch = 65
end_epochs = 85
for epoch in range(start_epoch, end_epochs, 1):
    model.train()  
    running_loss = 0.0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(trainloader)}")

print('Finished Training')


Epoch 66, Loss: 0.2330660503387102
Epoch 67, Loss: 0.23035166358228015
Epoch 68, Loss: 0.21941079401057215
Epoch 69, Loss: 0.21446371772473793
Epoch 70, Loss: 0.2143060831129551
Epoch 71, Loss: 0.20746653783350485
Epoch 72, Loss: 0.19423287330176972
Epoch 73, Loss: 0.19316404768379872
Epoch 74, Loss: 0.19349976320906426
Epoch 75, Loss: 0.18808665931990837
Epoch 76, Loss: 0.1835711264861282
Epoch 77, Loss: 0.1832585158279544
Epoch 78, Loss: 0.17482615417066263
Epoch 79, Loss: 0.1717983616445941
Epoch 80, Loss: 0.16325067849942135
Epoch 81, Loss: 0.16432994135217158
Epoch 82, Loss: 0.1608588536255047
Epoch 83, Loss: 0.15976943405225175
Epoch 84, Loss: 0.15816418080006406
Epoch 85, Loss: 0.15309612631638156
Finished Training


In [29]:
model.eval()
correct = 0
total = 0
predicted_labels = []
true_labels = []
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        # Forward pass
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        predicted_labels.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())
# accuracy
accuracy = correct / total
# F-score
from sklearn.metrics import f1_score
f_score = f1_score(true_labels, predicted_labels, average='weighted')
print(f"Accuracy: {accuracy}")
print(f"F-score: {f_score}")

Accuracy: 0.6608
F-score: 0.6605427644665073
